<a href="https://colab.research.google.com/github/tjturnage/multipage/blob/master/xmacis_dash.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install jupyter-dash -q
!pip install dash-bootstrap-components
import warnings
warnings.filterwarnings('ignore')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 16.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.7/219.7 KB 2.7 MB/s eta 0:00:00


In [2]:
import json
import requests
#from dataclasses import dataclass, field
#from typing import List
from jupyter_dash import JupyterDash  # pip install dash
from dash import dcc, html, Output, Input, no_update
import dash_bootstrap_components as dbc
import plotly.express as px
import math
from datetime import datetime
import pandas as pd
import numpy as np

parms_url = "http://data.rcc-acis.org/StnData?params="
meta_url = "http://data.rcc-acis.org/StnMeta"

class XMACIS():
    def __init__(self,station):

        self.station = station

         #grab metadata related to station's Period Of Record (POR)
        self.por_metadata = self.get_por_metadata()
        self.df = self.get_all_por_data()


    def get_por_metadata(self):
        """
        Returns list of strings: ['YYYY-mm-dd','YYYY-mm-dd']
        Refers to first/last dates ofperiod of record
        """
        params = {"sids": self.station, "elems": "maxt", "meta": ["name","valid_daterange"]}
        req = requests.post(meta_url,json=params)
        data = req.json()
        meta = data['meta'][0]
        self.full_name = meta['name']
        self.por_start = meta['valid_daterange'][0][0]
        self.por_start = '1975-01-01'
        self.por_end = meta['valid_daterange'][0][1]
        self.por_start = '1979-12-31'
        self.por_start_year = int(self.por_start[0:4])
        self.por_end_year = int(self.por_end[0:4])
        self.por_year_range = self.por_end_year - self.por_start_year + 1
        self.por_year_index = pd.date_range(start=f'1/1/{self.por_start_year}', periods=self.por_year_range, freq='Y')
        self.por_label = f'{self.por_start_year} - {self.por_end_year}'
        return

    def get_all_por_data(self):
        parms = {
        "sid":self.station,
        "sdate":self.por_start,
        "edate":self.por_end,
        "elems":[
            {"name":"maxt","interval":"dly","duration":"dly"},
            {"name":"mint","interval":"dly","duration":"dly"},
            {"name":"avgt","interval":"dly","duration":"dly"},
            ]}

        req = requests.post(parms_url,json=parms)
        data = req.json()
        df_init = pd.json_normalize(data)
        df_dp = df_init["data"].apply(pd.Series).T
        df = df_dp[0].apply(pd.Series)
        df.columns = ['dts','maxt','mint','avgt']
        df['date'] = pd.DatetimeIndex(df['dts'])
        df.set_index('date', inplace=True)
        df['year'] = df.index.year
        df['month'] = df.index.month
        df['day'] = df.index.day
        df = df.astype({"dts":'str', "year":'int', "month":'int', "day":'int', "maxt":'int', "mint":'int', "avgt":'float'}, errors='ignore')
        #df['month'] = df.index.strftime('%b'


        melted = pd.melt(df, id_vars=['dts','year','month','day'], value_vars=['mint', 'maxt', 'avgt'])
        melted = melted.astype({"variable": 'category'}, errors='ignore')
      
        return melted


test = XMACIS('GRRthr')
print(test.df)
print(test.df.dtypes)

output = test.df.to_csv(index=False,header=True)
#print(output)
# name of csv file 

with open('output-mmtr.csv', 'w') as outfile:
  for lines in output:
    outfile.write(lines)


              dts  year  month  day variable  value
0      1979-12-31  1979     12   31     mint   26.0
1      1980-01-01  1980      1    1     mint   26.0
2      1980-01-02  1980      1    2     mint   29.0
3      1980-01-03  1980      1    3     mint   24.0
4      1980-01-04  1980      1    4     mint   17.0
...           ...   ...    ...  ...      ...    ...
47248  2023-02-09  2023      2    9     avgt   38.0
47249  2023-02-10  2023      2   10     avgt   31.0
47250  2023-02-11  2023      2   11     avgt   32.0
47251  2023-02-12  2023      2   12     avgt   38.0
47252  2023-02-13  2023      2   13     avgt   35.0

[47253 rows x 6 columns]
dts           object
year           int64
month          int64
day            int64
variable    category
value        float64
dtype: object


In [3]:
from jupyter_dash import JupyterDash  # pip install dash
from dash import dcc, html, Output, Input, no_update
import dash_bootstrap_components as dbc
import plotly.express as px
import math
from datetime import datetime

now = datetime.utcnow()
THIS_YEAR = now.year        # Set Slider end year to equal current year
NEXT_YEAR = THIS_YEAR + 1   # ensures range command to build the slider includes this year

app = JupyterDash(__name__, external_stylesheets=[dbc.themes.CYBORG])


app.layout = html.Div([
    html.H4('Analysis of Iris data using scatter matrix'),
    dcc.Dropdown(
        id="dropdown",
        options=['2015', '2017'],
        value=[2015, 2017],
        multi=False
    ),
    dcc.Graph(id="graph"),
])



@app.callback(
    Output("graph", "fig"), 
    Input("dropdown", "val"))
def update_bar_chart(yr):
  #yr = int(val)
  dff = test.df.copy(deep=True)
  #dff_final = dff[dff['year'] == yr]
  #print(dff_final)
  fig = px.scatter(dff, x=dff.month, y=dff.value, color=dff.variable)
  #fig.update_layout(autotypenumbers='convert types')
  return fig


app.run_server(mode='inline', port=8030)



<IPython.core.display.Javascript object>